# Импотрты

In [ ]:
import random

import numpy as np
import pandas as pd

import torch

torch.cuda.empty_cache()

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

In [ ]:
# Фиксируем random seed
SEED = 42
seed_all(SEED)

In [ ]:
from pathlib import Path

DATA_PATH = Path('../../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_CACHE = DATA_PATH / Path('cache_dir/')
DATA_CACHE.mkdir(parents=True, exist_ok=True)

DATA_INPUT = DATA_PATH / Path('input/')
DATA_INPUT.mkdir(parents=True, exist_ok=True)

DATA_OUTPUT = DATA_PATH / Path('output/')
DATA_OUTPUT.mkdir(parents=True, exist_ok=True)

OFFLOAD_DIR = DATA_CACHE / Path('offload_weights/')
OFFLOAD_DIR.mkdir(parents=True, exist_ok=True)


import pandas as pd

pd.set_option('display.max_colwidth', 500) 

In [ ]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_path)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Название модели
MODEL_NAME = "deepseek-ai/deepseek-llm-7b-chat"

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir="./cache")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    offload_folder=OFFLOAD_DIR,
    cache_dir=DATA_CACHE
)

# Устанавливаем pad_token_id
model.generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

print("✅ Модель успешно загружена!")


In [ ]:
chat_history = []

def ask_model(user_input, max_tokens=512, temperature=0.1):
    global chat_history  # Используем глобальную историю

    # Добавляем сообщение пользователя в историю
    chat_history.append({"role": "user", "content": user_input})

    # Создаем input tensor с учетом всей истории
    input_tensor = tokenizer.apply_chat_template(
        chat_history, add_generation_prompt=True, return_tensors="pt"
    )

    # Генерация ответа
    outputs = model.generate(
        input_tensor.to(model.device),
        max_new_tokens=max_tokens,
        # temperature=temperature,
        # top_p=0.9,
        # repetition_penalty=1.1,
    )

    # Декодирование и обработка ответа
    response = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True).strip()

    # Добавляем ответ модели в историю
    chat_history.append({"role": "assistant", "content": response})

    return response  # Возвращаем ответ


In [ ]:
content = '''
Что это за документ? Определи по контексту:
'Соглашение о расторжении\nКонтракта № СТ-1383/22 от 31.01.2022 г.\nна поставку наборов для катетеризации на 2022 год\n\nпос. Первомайское «30» декабря 2022 г.\n\nГосударственное бюджетное учреждение здравоохранения Ленинградской ласти «Рощинская\nмежрайонная больница» (ГБУЗ ЛО «Рощинская МБ»), именуемое в дальнейшем «Заказчик», в лице\nтлавного врача Казарова Эрнеста Эдуардовича, действующего на основании Устава, Общество с\nограниченной ответственностью «Болюсмед» (ООО «Болюсмед»), именуемое в дальнейшем\n«Поставщик», в лице генерального директора Беловой Христины Витальевны, действующего на\nосновании Устава, с другой стороны, здесь и далее именуемые «Стороны», заключили настоящее\nСоглашение к Контракту № СТ-1383/22 от 31.01.2022 г. о нижеследующем:\n\n1. Руководствуясь частью 8 статьи 95 ФЗ №44-ФЗ «О контрактной системе в сфере закупок товаров, работ,\n‘услуг для обеспечения государственных и муниципальных нужд», пунктом 1 статьи 450 ГКРФ и пунктами\n11.2, 11.3. Контракта № СТ-1383/22 от 31.01.2022 г, Стороны договорились расторгнуть Контракт\n№ СТ-1383/22 от 31.01.2022 г. по соглашению сторон.\n\n2. Цена Контракта на момент расторжения соответствует стоимости фактически поставленного товара в\nсумме 108 150,00 руб. (Сто восемь тысяч сто пятьдесят рублей 00 копеек), НДС не облагается.\n\n3. Оплата за фактически поставленный товар произведена Заказчиком в соответствии с условиями\nКонтракта № СТ-1383/22 от 31.01.2022 г.\n\n4. Обязательства в оставшейся части на сумму 557 830,00 руб, (Пятьсот пятьдесят семь тысяч восемьсот\nтридцать рублей 00 копеек) считаются прекращенными, поставке и оплате не подлежат.\n\n5. Все остальные условия Контракта № СТ-1383/22 от 31.01.2022 г. не затронутые настоящим\nСоглашением, остаются неизменными и Стороны подтверждают по ним свои обязательства,\n\n6. Настоящее Соглашение составлено в двух подлинных экземплярах, по одному экземпляру для каждой из\nСторон, имеющих одинаковую юридическую силу, и является неотъемлемой частью Контракта\n№ СТ-1383/22 от 31.01.2022 г.\n\n7. Адреса, реквизиты и подписи сторон:\n\nЗаказчик: ГБУЗ ЛО «Рощинская МБ» Поставщик: ООО «Болюсмед»\nЮридический и почтовый адрес: 188855, Юридический адрес: 194295, г Санкт-Петербург,\nЛенинградская область, Выборгский район, ул. Ивана Фомина, д. 7, корп. 3, кв. 37\n\nп. Первомайское, ул. Ленина, д.54 «А» Почтовый адрес: 194064, г. Санкт-Петербург,\nТел: 8(81378)68-509, е-тай: гто@тьох.ги пр. Раевского, д. 14, корп. 2, литер А, пом.8-Н,\nИНН 4704047468 КПП 470401001 комн.4Б,5А,5Б (обособленное подразделение)\nОГРН 1034700879212 Тел: (812) 703-50-98, е-тай: т@Боаетед п\nБанковские реквизиты:\n\nОтделение Ленинградское Банка России// ИНН 7802664190 КПИ 780201001\n\nУФК по Ленинградской области г. Санкт-Петербур; ОГРН 1187847123947 от 27.04.2018\n\nКомитет финансов Ленинградской области Банковские реквизиты:\n\n(ГБУЗ ЛО «Рощинская МБ» л/с 2245620150) ПАО «Банк «Санкт-Петербург»\n\nБИК 014106101 БИК 044030790\n\nР/сч 03224643410000004500 Р/сч 40702810490700001210\n\nК/сч 40102810745370000006 К/сч 30101810900000000790\n\n_ Генеральный директор\nОз«Болюсмед»\nоо\n\n/ Х.В. Белова'
'''

In [ ]:
answer = ask_model(content)
print(f"🤖 AI: {answer}")

In [ ]:
content = '''
Составь список всех важных параметров
'''

In [ ]:
answer = ask_model(content)
print(f"🤖 AI: {answer}")